In [ ]:
import torch
import numpy as np

See tutorial: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

# TENSORS

## Tensor initialization

In [ ]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

In [ ]:
x_data

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

In [ ]:
shape = (2, 3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

## Tensor Attributes

In [ ]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

## Tensor Operations

See : https://pytorch.org/docs/stable/torch.html

In [ ]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
tensor = torch.ones(4, 4)
tensor[:,1] = 0
print(tensor)

In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

In [ ]:
# This computes the element-wise product
print(f"tensor.mul(tensor) \n {tensor.mul(tensor)} \n")
# Alternative syntax:
print(f"tensor * tensor \n {tensor * tensor}")

In [ ]:
# matrix multiplication
print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n")
# Alternative syntax:
print(f"tensor @ tensor.T \n {tensor @ tensor.T}")

In [ ]:
# In-place operations
print(tensor, "\n")
tensor.add_(5)
print(tensor)

# In-place operations save some memory, but can be problematic when computing derivatives because of an immediate loss of history. Hence, their use is discouraged.

In [ ]:
tensor

## Bridge with NumPy

In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

In [ ]:
n = np.ones(5)
t = torch.from_numpy(n)

In [ ]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

# AUTOGRAD

## Usage in PyTorch

In [ ]:
# import torch
# from torchvision.models import resnet18, ResNet18_Weights
# model = resnet18(weights=ResNet18_Weights.DEFAULT)
# data = torch.rand(1, 3, 64, 64)
# labels = torch.rand(1, 1000)

## Differentiation in Autograd

See a nice tutorial here: https://www.youtube.com/watch?v=MswxJw-8PvE&ab_channel=ElliotWaite

In [ ]:
#  'requires_grad=True' signals to autograd that every operation on them should be tracked.
a = torch.tensor([2., 3., 5.], requires_grad=True)
b = torch.tensor([6., 4., 7.], requires_grad=True)

In [ ]:
Q = 3*a**3 - b**2

In [ ]:
external_grad = torch.tensor([1., 1., 1.])
Q.backward(gradient=external_grad)

In [ ]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

---

In [ ]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

f = a*b

In [ ]:
f.backward()

In [ ]:
print(a.grad)
print(b.grad)

---

In [ ]:
a = torch.tensor([2.0, 4.0], requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

f = 9*a
f.backward(gradient=torch.ones(2))

a.grad

# Application to Wirings

In [ ]:
import non_local_boxes

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
torch.zeros( (2, 4, 4, 32) ).shape

In [ ]:
non_local_boxes.utils.CHSH_flat

In [ ]:
torch.rand((3, 2))

In [ ]:
torch.randint(2, (3,2))

In [ ]:
W = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns)
P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR


#non_local_boxes.evaluate.R(W, P, Q)[:,:,:,:,0] == non_local_boxes.utils.matrix_to_tensor(P)
non_local_boxes.evaluate.phi_flat(W, P, Q)

In [ ]:
W = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns)
P = non_local_boxes.utils.PR
Q = non_local_boxes.utils.SR

W_np = non_local_boxes.utils_with_numpy.W_BS09(non_local_boxes.evaluate.nb_columns)
P_np = non_local_boxes.utils_with_numpy.PR
Q_np = non_local_boxes.utils_with_numpy.SR

In [ ]:
T1 = torch.tensordot(non_local_boxes.evaluate.A1, W, dims=1) + non_local_boxes.evaluate.A2
T2 = T1.repeat(2, 1, 1, 1, 1)
#T2 = torch.reshape(T2, (2,2,4,4,-1))
T3 = torch.transpose(T2, 0, 1)

T1_np = np.tensordot(non_local_boxes.evaluate_with_numpy.A1, W_np, axes=([3, 0])) + non_local_boxes.evaluate_with_numpy.A2
T2_np = np.tensordot(np.ones((2)), T1_np, axes=0)
T3_np = np.transpose(T2_np, (1,0,2,3,4))

print(T3.numpy()==T3_np)
# print(T1.shape)
# print(T3.shape)
#print(T2_np)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X = np.array([[1,3], [4,1], [4, 5]])

triangle = plt.Polygon(X[:3,:], color="snow")
plt.gca().add_patch(triangle)

number_steps = 50
threshold = (3 + np.sqrt(6))/6
number_products =10

W = non_local_boxes.utils.W_BS09(non_local_boxes.evaluate.nb_columns)

for i in range(number_steps+1):
    for j in range(number_steps-i+1):
        alpha = i/number_steps
        beta = j/number_steps
        P = alpha*non_local_boxes.utils.PR + beta*non_local_boxes.utils.P_0 + (1-alpha-beta)*non_local_boxes.utils.P_1   # P is a 4x4 matrix
        color_point = "orangered"
        
        Q=torch.clone(P)
        Q=non_local_boxes.utils.matrix_to_tensor(Q)  # Q is a 2x2x2x2 tensor 
        for k in range(number_products+1):
            if non_local_boxes.evaluate.h_flat(Q)[0] > threshold:
                color_point = (0, 0.1*(1-k/number_products)+1*(k/number_products), 0.1*(1-k/number_products)+1*(k/number_products))
                break
            #Q2=Q.copy()
            Q=non_local_boxes.evaluate.R(W, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]

        plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                

plt.show()

# Differentiate the function phi_flat

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ], requires_grad=True)
W = torch.reshape(torch.kron(torch.ones(non_local_boxes.evaluate.nb_columns), W), (32, -1))

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR

external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)

In [ ]:
non_local_boxes.evaluate.phi_flat(W, P, Q).backward(gradient=external_grad)

In [ ]:
print(W.grad)

In [ ]:
print(W[0,0].grad)

In [ ]:
W.is_leaf

---

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ], requires_grad=True)

f = 2*W
f.backward(gradient=torch.ones(32))

W.grad

---

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ], requires_grad=True)
W2 = torch.t(W.repeat(non_local_boxes.evaluate.nb_columns, 1))

f = 2*W2
f.backward(gradient=torch.ones(32,non_local_boxes.evaluate.nb_columns))

W.grad

---

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ], requires_grad=True)
W2 = torch.t(W.repeat(non_local_boxes.evaluate.nb_columns, 1))

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR

external_grad = torch.ones(2, 2, non_local_boxes.evaluate.nb_columns, 4, 4)
non_local_boxes.evaluate.A(W2).backward(gradient=external_grad)

In [ ]:
W.requires_grad, W.is_leaf, W2.requires_grad, W2.is_leaf 

In [ ]:
W.grad

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ], requires_grad=True)
W2 = torch.t(W.repeat(non_local_boxes.evaluate.nb_columns, 1))

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR

external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
non_local_boxes.evaluate.phi_flat(W2, P, Q).backward(gradient=external_grad)

In [ ]:
W.grad

---

In [ ]:
W = torch.rand((32, non_local_boxes.evaluate.nb_columns), requires_grad=True)

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR

external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
non_local_boxes.evaluate.phi_flat(W, P, Q).backward(gradient=external_grad)

In [ ]:
W.grad

---

In [ ]:
W = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR

external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)
non_local_boxes.evaluate.phi_flat(W, P, Q).backward(gradient=external_grad)

In [ ]:
W.grad